# Exploratory data analysis (EDA)

Use this notebook to explore and visualize the data.

In [3]:
import os
import ibis
import ibis.selectors as s

import plotly.express as px

from icarus.config import (
    DATA_DIR,
    RAW_DATA_DIR,
    BUY_SELL_TABLE,
    SOCIAL_MEDIA_TABLE,
)
from icarus.catalog import Catalog
from icarus.synthetic_data.investments import fake, seed_table

px.defaults.template = "plotly_dark"
ibis.options.interactive = True

In [4]:
f"{10_000_000_000 / (60 * 60 * 24):,.2f}"

'115,740.74'

In [5]:
lookback = ibis.interval(days=1)
step = ibis.interval(seconds=1)

In [6]:
t = (
    ibis.range(ibis.now() - lookback, ibis.now(), step=step)
    .unnest()
    .name("timestamp")
    .as_table()
    .mutate(
        index=(ibis.row_number().over(order_by="timestamp")),
        symbol=ibis.random(),
        bid=ibis.random(),
        bid_size=ibis.random(),
        ask=ibis.random(),
        ask_size=ibis.random(),
        **{c: 2 * (ibis.random() - 0.5) for c in ["a", "b", "c"]},
        **{
            c: ibis.null(type=str)
            for c in list(map(chr, range(ord("d"), ord("z") + 1)))
        },
    )
    .relocate("index", "timestamp")
)
t

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ index ┃ timestamp               ┃ symbol   ┃ bid      ┃ bid_size ┃ ask      ┃ ask_size ┃ a         ┃ b         ┃ c         ┃ d      ┃ e      ┃ f      ┃ g      ┃ h      ┃ i      ┃ j      ┃ k      ┃ l      ┃ m      ┃ n      ┃ o      ┃ p      ┃ q      ┃ r      ┃ s      ┃ t      ┃ u      ┃ v      ┃ w      ┃ x      ┃ y      ┃ z      ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ int64 │ timestamp               │ float64  │ float64  │ float64  │ float64  │ float64  │ float64   │ float64   │ float64   │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │
├───────┼─────────────────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│     0 │ 2024-08-15 03:01:59.512 │ 0.941685 │ 0.060818 │ 0.057285 │ 0.800820 │ 0.992450 │ -0.381563 │  0.673071 │  0.276644 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     1 │ 2024-08-15 03:02:00.512 │ 0.661545 │ 0.112700 │ 0.557890 │ 0.613913 │ 0.115345 │ -0.241797 │  0.770039 │  0.795991 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     2 │ 2024-08-15 03:02:01.512 │ 0.106904 │ 0.417297 │ 0.507067 │ 0.846771 │ 0.698977 │  0.777133 │ -0.792764 │  0.280758 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     3 │ 2024-08-15 03:02:02.512 │ 0.403178 │ 0.642852 │ 0.323979 │ 0.053536 │ 0.083531 │  0.563244 │ -0.389257 │  0.711820 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     4 │ 2024-08-15 03:02:03.512 │ 0.201227 │ 0.374165 │ 0.772471 │ 0.683315 │ 0.840357 │  0.224795 │  0.580763 │ -0.070744 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     5 │ 2024-08-15 03:02:04.512 │ 0.733130 │ 0.813511 │ 0.867238 │ 0.237229 │ 0.861680 │ -0.953303 │  0.880240 │ -0.008706 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     6 │ 2024-08-15 03:02:05.512 │ 0.750758 │ 0.542480 │ 0.694365 │ 0.255499 │ 0.748122 │ -0.763058 │  0.218854 │  0.327044 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ N

In [7]:
import random

symbols = [chr(i) for i in range(ord("A"), ord("Z") + 1)]
symbols = [
    f"{a}{b}{c}{d}" for a in symbols for b in symbols for c in symbols for d in symbols
]
symbols = random.sample(symbols, 500)
symbols[:5]

['KFBH', 'IUFR', 'SLNK', 'ZTOT', 'EIPA']

In [8]:
symbols_case = ibis.case()
for i, symbol in enumerate(symbols):
    symbols_case = symbols_case.when(
        (i / len(symbols) < ibis._["symbol"])
        & (ibis._["symbol"] < ((i + 1) / len(symbols))),
        symbol,
    )
symbols_case = symbols_case.else_(ibis.null()).end()

In [9]:
t = t.mutate(
    symbol=symbols_case,
    bid_size=(ibis._["bid_size"] * 1000).cast("int64"),
    ask_size=(ibis._["ask_size"] * 1000).cast("int64"),
)
t

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ index ┃ timestamp               ┃ symbol ┃ bid      ┃ bid_size ┃ ask      ┃ ask_size ┃ a         ┃ b         ┃ c         ┃ d      ┃ e      ┃ f      ┃ g      ┃ h      ┃ i      ┃ j      ┃ k      ┃ l      ┃ m      ┃ n      ┃ o      ┃ p      ┃ q      ┃ r      ┃ s      ┃ t      ┃ u      ┃ v      ┃ w      ┃ x      ┃ y      ┃ z      ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ int64 │ timestamp               │ string │ float64  │ int64    │ float64  │ int64    │ float64   │ float64   │ float64   │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │
├───────┼─────────────────────────┼────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│     0 │ 2024-08-15 03:02:00.570 │ UBRE   │ 0.249286 │      783 │ 0.242910 │      839 │ -0.775910 │  0.224334 │ -0.245893 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     1 │ 2024-08-15 03:02:01.570 │ QYBU   │ 0.217163 │      781 │ 0.686105 │      915 │  0.409511 │  0.258960 │ -0.223681 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     2 │ 2024-08-15 03:02:02.570 │ EKXC   │ 0.503741 │      831 │ 0.731467 │       25 │ -0.091764 │ -0.373889 │ -0.363401 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     3 │ 2024-08-15 03:02:03.570 │ URUQ   │ 0.534424 │       75 │ 0.641017 │      575 │  0.416319 │  0.283495 │  0.672193 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     4 │ 2024-08-15 03:02:04.570 │ VHVU   │ 0.719378 │      305 │ 0.585454 │      292 │  0.185710 │ -0.240103 │ -0.313028 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     5 │ 2024-08-15 03:02:05.570 │ IPVF   │ 0.971505 │      490 │ 0.762250 │      270 │  0.247360 │  0.482406 │  0.887382 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     6 │ 2024-08-15 03:02:06.570 │ BCPA   │ 0.796209 │      375 │ 0.761442 │       47 │ -0.940216 │ -0.834858 │  0.433293 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     7 │ 2024-0

In [10]:
t

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ index ┃ timestamp               ┃ symbol ┃ bid      ┃ bid_size ┃ ask      ┃ ask_size ┃ a         ┃ b         ┃ c         ┃ d      ┃ e      ┃ f      ┃ g      ┃ h      ┃ i      ┃ j      ┃ k      ┃ l      ┃ m      ┃ n      ┃ o      ┃ p      ┃ q      ┃ r      ┃ s      ┃ t      ┃ u      ┃ v      ┃ w      ┃ x      ┃ y      ┃ z      ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ int64 │ timestamp               │ string │ float64  │ int64    │ float64  │ int64    │ float64   │ float64   │ float64   │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │
├───────┼─────────────────────────┼────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│     0 │ 2024-08-15 03:02:00.993 │ JEJL   │ 0.107446 │      956 │ 0.604128 │      229 │ -0.552863 │ -0.838911 │  0.705470 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     1 │ 2024-08-15 03:02:01.993 │ EOBD   │ 0.070317 │       78 │ 0.736908 │      481 │ -0.782693 │  0.465634 │ -0.051302 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     2 │ 2024-08-15 03:02:02.993 │ OZOT   │ 0.147363 │      876 │ 0.666978 │       69 │  0.451983 │  0.760876 │ -0.030605 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     3 │ 2024-08-15 03:02:03.993 │ WRIT   │ 0.223614 │      424 │ 0.884469 │       94 │ -0.273077 │  0.049044 │ -0.253167 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     4 │ 2024-08-15 03:02:04.993 │ MJML   │ 0.417077 │      305 │ 0.343850 │      258 │  0.905540 │  0.053693 │  0.894668 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     5 │ 2024-08-15 03:02:05.993 │ TIYA   │ 0.068448 │      746 │ 0.910013 │      172 │  0.143857 │  0.242721 │ -0.038548 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     6 │ 2024-08-15 03:02:06.993 │ LDAU   │ 0.648114 │      358 │ 0.830861 │      514 │  0.531712 │ -0.438674 │ -0.338066 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     7 │ 2024-0

In [11]:
seed_table

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ ticker ┃ sector  ┃ name                                     ┃ base_price ┃ shares_total ┃ trend  ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━┩
│ string │ string  │ string                                   │ float64    │ int64        │ string │
├────────┼─────────┼──────────────────────────────────────────┼────────────┼──────────────┼────────┤
│ AAI    │ tech    │ Athena's AI                              │      100.0 │     10000000 │ down   │
│ SUBS   │ tech    │ SubWay Cloud                             │      150.0 │    100000000 │ up     │
│ II     │ tech    │ Icarus Investments                       │     3000.0 │       500000 │ flat   │
│ IBIS   │ service │ Interior Basics In Season                │     1000.0 │      1000000 │ down   │
│ VODA   │ service │ Voyage's Offices, Doors, and Accessories │     2500.0 │      1000000 │ up     │
└────────┴─────────┴──────────────────────────────────────────┴────────────┴──────────────┴────────┘

In [12]:
catalog = Catalog()
catalog.list_tables()

['buy_sell.delta', 'social_media.delta']

In [13]:
seed_table

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ ticker ┃ sector  ┃ name                                     ┃ base_price ┃ shares_total ┃ trend  ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━┩
│ string │ string  │ string                                   │ float64    │ int64        │ string │
├────────┼─────────┼──────────────────────────────────────────┼────────────┼──────────────┼────────┤
│ AAI    │ tech    │ Athena's AI                              │      100.0 │     10000000 │ down   │
│ SUBS   │ tech    │ SubWay Cloud                             │      150.0 │    100000000 │ up     │
│ II     │ tech    │ Icarus Investments                       │     3000.0 │       500000 │ flat   │
│ IBIS   │ service │ Interior Basics In Season                │     1000.0 │      1000000 │ down   │
│ VODA   │ service │ Voyage's Offices, Doors, and Accessories │     2500.0 │      1000000 │ up     │
└────────┴─────────┴──────────────────────────────────────────┴────────────┴──────────────┴────────┘

In [14]:
data_glob = os.path.join(DATA_DIR, RAW_DATA_DIR, BUY_SELL_TABLE, "*.parquet")
bs = ibis.read_parquet(data_glob)
bs

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ticker ┃ buy_sell                                                                         ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string │ array<struct<timestamp: timestamp(6), type: string, shares: float64, price: flo… │
├────────┼──────────────────────────────────────────────────────────────────────────────────┤
│ AAI    │ [{...}, {...}, ... +126]                                                         │
│ SUBS   │ [{...}, {...}, ... +126]                                                         │
│ II     │ [{...}, {...}, ... +126]                                                         │
│ IBIS   │ [{...}, {...}, ... +126]                                                         │
│ VODA   │ [{...}, {...}, ... +126]                                                         │
│ AAI    │ [{...}, {...}, ... +126]                                                         │
│ SUBS   │ [{...}, {...}, ... +126]                                                         │
│ II     │ [{...}, {...}, ... +126]                                                         │
│ IBIS   │ [{...}, {...}, ... +126]                                                         │
│ VODA   │ [{...}, {...}, ... +126]                                                         │
│ …      │ …                                                                                │
└────────┴──────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
bs = catalog.table("buy_sell")
bs

In [ ]:
t = bs.group_by("ticker").agg(count=ibis._.count()).order_by(ibis.desc("count"))
t

In [ ]:
px.bar(
    t,
    x="ticker",
    y="count",
    title="Number of trades per symbol",
    color="ticker",
)

In [ ]:
bs.count()

In [ ]:
bs["location"].value_counts()

In [ ]:
f"{bs.count().to_pyarrow().as_py():,}"

In [ ]:
data_glob = os.path.join(DATA_DIR, RAW_DATA_DIR, SOCIAL_MEDIA_TABLE, "*.parquet")
sm = ibis.read_parquet(data_glob)
sm

In [ ]:
sm = catalog.table("social_media")
sm